![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

In [339]:
import pandas as pd
import numpy as np

# Start coding here...
bank_df = pd.read_csv("bank_marketing.csv")
bank_df.head(n=10)

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,day,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,13,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,19,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,23,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,27,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,3,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
5,5,45,services,married,basic.9y,unknown,no,no,telephone,may,5,198,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
6,6,59,admin.,married,professional.course,no,no,no,telephone,may,3,139,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
7,7,41,blue-collar,married,unknown,unknown,no,no,telephone,may,12,217,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
8,8,24,technician,single,professional.course,no,yes,no,telephone,may,21,380,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
9,9,25,services,single,high.school,no,yes,no,telephone,may,5,50,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Client DataFrame

In [340]:
client_df = bank_df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']]
client_df = client_df.rename(columns = {'client_id': 'id'})
client_df.head()

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no
3,3,40,admin.,married,basic.6y,no,no,no
4,4,56,services,married,high.school,no,no,yes


In [341]:
client_df['education'] = client_df['education'].apply(lambda x: x.replace(".", "-"))
client_df['education'].replace(to_replace = "unknown", value = np.NaN, inplace=True)
client_df.head()

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic-4y,no,no,no
1,1,57,services,married,high-school,unknown,no,no
2,2,37,services,married,high-school,no,yes,no
3,3,40,admin.,married,basic-6y,no,no,no
4,4,56,services,married,high-school,no,no,yes


In [342]:
import re

client_df['job'] = client_df['job'].apply(lambda x: re.sub(".$", "", x))
client_df.head()

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemai,married,basic-4y,no,no,no
1,1,57,service,married,high-school,unknown,no,no
2,2,37,service,married,high-school,no,yes,no
3,3,40,admin,married,basic-6y,no,no,no
4,4,56,service,married,high-school,no,no,yes


In [343]:
client_df.to_csv('client.csv', index = False)

### Campaign DataFrame

In [344]:
campaign_df = bank_df[['client_id', 'campaign','duration', 'pdays', 'previous', 'poutcome', 'y', 'month', 'day']]

campaign_df = campaign_df.rename(columns = {'campaign': 'number_contacts', 'duration': 'contact_duration', 'previous': 'previous_campaign_contacts', 'poutcome': 'previous_outcome', 'y': 'campaign_outcome'})

campaign_df['campaign_id'] = 1

campaign_df['year'] = 2022

campaign_df.head()

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,month,day,campaign_id,year
0,0,1,261,999,0,nonexistent,no,may,13,1,2022
1,1,1,149,999,0,nonexistent,no,may,19,1,2022
2,2,1,226,999,0,nonexistent,no,may,23,1,2022
3,3,1,151,999,0,nonexistent,no,may,27,1,2022
4,4,1,307,999,0,nonexistent,no,may,3,1,2022


In [345]:
campaign_df['last_contact_date'] = pd.to_datetime(campaign_df.year.astype(str) + campaign_df.month.astype(str) + campaign_df.day.astype(str), format = '%Y%b%d').dt.date
campaign_df = campaign_df.drop(['year', 'month', 'day'], axis = 1)
campaign_df.head()

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,campaign_id,last_contact_date
0,0,1,261,999,0,nonexistent,no,1,2022-05-13
1,1,1,149,999,0,nonexistent,no,1,2022-05-19
2,2,1,226,999,0,nonexistent,no,1,2022-05-23
3,3,1,151,999,0,nonexistent,no,1,2022-05-27
4,4,1,307,999,0,nonexistent,no,1,2022-05-03


In [346]:
campaign_df['previous_outcome'] = campaign_df['previous_outcome'].map({'success': 1, 'failure': 0, 'nonexistent': np.NaN})

campaign_df['campaign_outcome'] = campaign_df['campaign_outcome'].map({'yes': 1, 'no': 0})

print(campaign_df['previous_outcome'].unique())
print(campaign_df['campaign_outcome'].unique())

[nan  0.  1.]
[0 1]


In [347]:
campaign_df.to_csv('campaign.csv', index = False)

### Economics DataFrame

In [348]:
economics_df = bank_df[['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed']]
economics_df = economics_df.rename(columns = {'euribor3m': 'euribor_three_months', 'nr_employed': 'number_employed'})
economics_df.head()

,client_id,emp_var_rate,cons_price_idx,euribor_three_months,number_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0


In [349]:
economics_df.to_csv('economics.csv', index = False)

### Client SQL Table

In [350]:
client_table = """
CREATE TABLE client(
    id serial PRIMARY KEY,
    age INT,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

In [351]:
campaign_table = """
CREATE TABLE campaign(
    campaign_id serial PRIMARY KEY,
    client_id serial REFERENCES client (id),
    number_contacts INT,
    contact_duration INT,
    pdays INT,
    previous_campaign_contacts INT,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE,        
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

In [352]:
economics_table = """
CREATE TABLE economics(
    client_id serial REFERENCES client (id),
    emp_var_rate FLOAT(3),
    cons_price_idx FLOAT(3),
    euribor_three_months FLOAT(3),
    number_employed FLOAT(3),
    PRIMARY KEY (client_id)
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""